In [ ]:
import os, cv2
import numpy as np
from os import listdir
from os.path import isfile, join
from PIL import Image
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import shutil
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
import keras.optimizers
import random
import scipy
import math
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
# CHANGE THIS to the path on your machine
projectDir = r'C:\Users\Viktor\Skola\embedded\embedded_project'

In [ ]:
imgFileDir = os.path.join(projectDir,'Stanford40_JPEGImages\JPEGImages')
CImagesDir = os.path.join(projectDir,'Stanford40_JPEGImages\CImages')
parentImgFileDir = os.path.join(projectDir,'Stanford40_JPEGImages')
xmlFileDir = os.path.join(projectDir,'Stanford40_XMLAnnotations\XMLAnnotations')

In [ ]:
def extractClassLabels(xmlFileDir):
    os.chdir(xmlFileDir)
    xmlFileNames = [f for f in listdir(xmlFileDir) if isfile(join(xmlFileDir, f))]
    #xmlFileNames = ['3m1.xml','3m2.xml','3m3.xml','3m4.xml']
    classLabels = []
    imageNames = []
    count = 1
    for file in xmlFileNames:
        root = ET.parse(file).getroot()
        filename = root.find('filename').text
        action = root.find('object/action').text
        classLabels.append(action)
        imageNames.append(filename)
        print(str(count)+'/'+str(len(xmlFileNames)), end='\r')
        count +=1
    return np.array(classLabels), np.array(imageNames)

In [ ]:
classList, allImageNames = extractClassLabels(xmlFileDir)


In [ ]:
(classUnique,counts) = np.unique(classList, return_counts=True)

(_,inverse) = np.unique(classList, return_inverse=True)

y = to_categorical(inverse,len(classUnique))

In [ ]:
batch_size = 32
img_height = 90
img_width = 90
actionImagesDir = os.path.join(projectDir,'Stanford40_JPEGImages\ActionImages')
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    actionImagesDir,
    validation_split=0.2,
    label_mode = 'categorical',
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    actionImagesDir,
    validation_split=0.2,
    label_mode = 'categorical',
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

In [ ]:
normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))

In [ ]:
#AUTOTUNE = tf.data.experimental.AUTOTUNE
#train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
def best_model(nrOfClasses):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(90 ,90, 3)))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(nrOfClasses, activation='softmax'))
    opt = keras.optimizers.Adam(0.0001)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuned_model = best_model(len(classUnique))

In [ ]:
def define_model(nrOfClasses, hyperparameters):
    model = Sequential()
    
    model.add(Conv2D(hyperparameters["nrOfFiltersInConvLayers"][0], (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(90 ,90, 3)))
    
    for nrOfFilters in hyperparameters["nrOfFiltersInConvLayers"][1:]:
        model.add(Conv2D(nrOfFilters, (3, 3), activation='relu', kernel_initializer='he_uniform'))
        model.add(MaxPooling2D((2, 2)))
        model.add(Dropout(hyperparameters["dropout"]))

    model.add(Flatten())
    
    for nrOfNeurons in hyperparameters["nrOfNeuronsInHiddenLayers"]:
        model.add(Dense(nrOfNeurons, activation='relu', kernel_initializer='he_uniform'))
        
    model.add(Dense(nrOfClasses, activation='softmax'))
    # compile model¨
    opt = keras.optimizers.Adam(learning_rate=hyperparameters["learningRate"])
    #opt = SGD(lr=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    #model.summary()
    return model

In [ ]:
nrOfConvLayers = [2,3,4]
nrOfFilters = [16,32,64,128]
nrOfHiddenLayers = [2,3,4]
nrOfHiddenLayerNeurons = [32,64,128,256,512]
dropout = [0.2,0.5,0.8]
learningRate = [0.00001,0.0001,0.001]

tests = 100

bestAccuracy = 0
bestHyperparameters = None

for test in range(tests):
    hyperparameters = {
        'nrOfFiltersInConvLayers': [],
        'nrOfNeuronsInHiddenLayers':[],
        'dropout': random.choice(dropout),
        'learningRate':random.choice(learningRate)
    }
    
    for layer in range(random.choice(nrOfConvLayers)):
        hyperparameters['nrOfFiltersInConvLayers'].append(random.choice(nrOfFilters))
    
    
    for layer in range(random.choice(nrOfHiddenLayers)):
        hyperparameters['nrOfNeuronsInHiddenLayers'].append(random.choice(nrOfHiddenLayerNeurons))
                       
    print(hyperparameters)
    
    model = define_model(len(classUnique),hyperparameters)
    model.fit(train_ds, epochs=20)
    result = model.evaluate(test_ds, batch_size=32)
    accuracy = result[1]
    
    if accuracy > bestAccuracy:
        bestAccuracy = accuracy
        bestHyperparameters = hyperparameters
    
    print(bestAccuracy)
    print(bestHyperparameters)

In [ ]:
tuned_model.fit(train_ds, epochs=20)
result = tuned_model.evaluate(test_ds, batch_size=32)

In [ ]:
result

In [ ]:
parentModelDir = os.path.join(projectDir,'code')
os.chdir(parentModelDir)
savedModelDir = os.path.join(parentModelDir, "saved_model")
if os.path.exists(savedModelDir):
    print("it exist")
else:
    print("does not exist, create folder")
    os.mkdir(savedModelDir)

tuned_model.save('saved_model/my_model')

In [ ]:
#convert to tensorflow lite model
# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model(savedModelDir+'/my_model') # path to the SavedModel directory
tflite_model = converter.convert()

os.makedirs(os.path.join(os.getcwd(),'saved_tflite_models'), exist_ok = True)
# Save the model.
with open('./saved_tflite_models/model_1.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
#load tensorflow lite model
interpreter = tf.lite.Interpreter(model_path='./saved_tflite_models/model_1.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()


In [ ]:
input_details

In [ ]:
ds_batches = tfds.as_numpy(test_ds)

In [ ]:
count_images = 0
for batch in ds_batches:
    for image in batch[1]:
        count_images +=1


In [ ]:
count_images

In [ ]:
xTest = np.zeros((count_images, 90,90,3)).astype('float32')
yTest = np.zeros((count_images,40)).astype('float32')

batch_number = 0
for batch in ds_batches:
    for image_number in range(len(batch[0])):
        xTest[batch_number*32+image_number] = batch[0][image_number]
        yTest[batch_number*32+image_number] = batch[1][image_number]
    batch_number +=1

        

In [ ]:
correct_predict = 0
for image_number in range(len(xTest)): 
    interpreter.set_tensor(input_details[0]['index'], xTest[image_number][np.newaxis, ...])
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    if (output_data[0] == yTest[image_number]).all():
        correct_predict +=1

In [ ]:
accuracy = correct_predict/count_images
accuracy